In [19]:
import pandas as pd
import numpy as np
import seaborn as sns

from collections import defaultdict, Counter

# CCLE name to Achilles ID mapping

Cell lines can be identified both by a CCLE name (for instance, "HELA_CERVIX") or with an Achilles ID (for instance, "ACH-001086"). For consistency, we will use the Achilles ID.

In [16]:
cell_line_info = pd.read_csv("../data/Cell_lines_annotations_20181226.txt",sep="\t")
ccle_achilles_map = dict(zip(cell_line_info["CCLE_ID"],cell_line_info["depMapID"]))

# Gencode v19 definitions

The gene expression estimates in the CCLE were computed using the GENCODE v19 definitions. Here we read these in and do some pre-formatting to extract the gene transcripts and ENSEMBL IDs, before saving to an HDF5 file for faster loading.

In [8]:
g19_definitions = pd.read_csv("../data/gencode.v19.genes.v7_model.patched_contigs.gtf.gz",sep="\t",skiprows=6,
                              names=["chrom","source","type","start","end",".1","strand",".2","info"])

# ENSEMBL gene ID
g19_definitions["ensembl_gene_id"] = g19_definitions["info"].apply(
    lambda x: x.split(";")[0][9:-1])

# Gene name (HUGO)
g19_definitions["gene_name"] = g19_definitions["info"].apply(
    lambda x: x.split("gene_name")[1].split(";")[0][2:-1])

# ENSEMBL transcript ID
g19_definitions["ensembl_tx_id"] = g19_definitions["info"].apply(
    lambda x: x.split("transcript_id")[1].split(";")[0][2:-1])

# ENSEMBL name map
ensembl_id_map = dict(zip(g19_gene_definitions["ensembl_gene_id"],g19_gene_definitions["gene_name"]))
ensembl_id_map = defaultdict(str, ensembl_id_map)

# Export to HDF5
g19_definitions.to_hdf("../data/gencode.v19.genes.v7_model.patched_contigs.h5",key="g19_definitions")

/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/pandas/core/generic.py:2531: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['chrom', 'source', 'type', '.1', 'strand', '.2', 'info', 'ensembl_gene_id', 'gene_name', 'ensembl_tx_id']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


# RNA-seq

The CCLE includes 1,019 cell lines for which deep RNA-sequencing was performed. These were then used to estimate gene expression, transcript expression, and exon inclusion ratios. Here we load these files and save to HDF5 for faster loading.

## Exon inclusions

In [ ]:
# Change the exon naming scheme
def reorder_exon(exon):
    exon_split = exon.split("_")
    return "_".join(exon_split[3:]) + "_" + "_".join(exon_split[:3])

exonusage = pd.read_csv("raw/ccle/CCLE_RNAseq_ExonUsageRatio_20180929.gct",skiprows=2,index_col=0,sep="\t")
exonusage.index = exonusage.index.map(lambda x:reorder_exon(x))

# Fix inconsistant NaN values
exonusage[exonusage=="\tNA"] = np.nan
exonusage[exonusage=="    NA"] = np.nan
exonusage[exonusage=="     NA"] = np.nan
exonusage = exonusage.astype(float)

# Define columns
exon_ids = pd.Series(exonusage.index,index=exonusage.index) + "_" + pd.Series(exonusage["gene_id"])
exonusage = exonusage.set_index(exon_ids).iloc[:,1:]
exonusage = exonusage.T

# Map CCLE cell line names to Achilles IDs
exonusage.index = exonusage.index.map(lambda x: ccle_achilles_map[x])

# Exon inclusion measurements require that the gene is sufficiently expressed.
# In the interest of space, we only keep exons with less than 800 missing values
exonusage_nans = exonusage.isna().sum(axis=0)
sns.distplot(exonusage_nans)
exonusage = exonusage[exonusage.columns[exonusage_nans<800]]

# We also drop exons with low variability of inclusion (standard deviation less than 0.1)
exonusage_stdevs = exonusage.std(axis=0)
sns.distplot(exonusage_stdevs)
exonusage = exonusage[exonusage.columns[exonusage_stdevs>0.1]]

# Export to HDF5
exonusage.to_hdf("../data/CCLE_RNAseq_ExonUsageRatio_20180929.hdf",key="exonusage")

## Transcript expression

In [ ]:
ccle_transcripts = pd.read_csv("../data/CCLE_RNAseq_rsem_transcripts_tpm_20180929.txt",sep="\t",index_col=1)
ccle_transcripts["gene_id"] = ccle_transcripts["gene_id"].apply(lambda x: ensembl_id_map[x])

# Change the transcript naming format
gene_transcript_ids = ccle_transcripts["gene_id"] + "_" + pd.Series(ccle_transcripts.index,index=ccle_transcripts.index)
ccle_transcripts = ccle_transcripts.set_index(gene_transcript_ids)

# Pseudo-log transform and name mapping
ccle_transcripts = ccle_transcripts.iloc[:,1:]
ccle_transcripts = np.log2(ccle_transcripts+1)
ccle_transcripts = ccle_transcripts.T
ccle_transcripts.index = ccle_transcripts.index.map(lambda x: ccle_achilles_map[x])

# Standard deviation filtering
ccle_transcript_stdevs = ccle_transcripts.std(axis=0)
sns.distplot(ccle_transcript_stdevs)
ccle_transcripts = ccle_transcripts[ccle_transcripts.columns[ccle_transcript_stdevs>0.25]]

# Export to HDF5
ccle_transcripts.to_hdf("../data/CCLE_RNAseq_rsem_transcripts_tpm_20180929.hdf",key="ccle_transcripts")

## Gene expression

In [ ]:
ccle_genex = pd.read_csv("../data/CCLE_RNAseq_rsem_genes_tpm_20180929.txt",sep="\t",index_col=0)

# Drop info columns
ccle_genex = ccle_genex.iloc[:,1:]

# Map ensembl IDs
ccle_gene_names = ccle_genex.index.map(lambda x: ensembl_id_map[x])
gene_names_ids = ccle_gene_names + "_" + pd.Series(ccle_genex.index,index=ccle_genex.index)
ccle_genex = ccle_genex.set_index(gene_names_ids)

# Pseudo-log transform and Achilles name map
ccle_genex = np.log2(ccle_genex+1)
ccle_genex = ccle_genex.T
ccle_genex.index = ccle_genex.index.map(lambda x: ccle_achilles_map[x])

# Export to HDF5
ccle_genex.to_hdf("../data/CCLE_RNAseq_rsem_genes_tpm_20180929.hdf",key="ccle_genex")

# RRBS

The CCLE includes reduced-representation bisulfite sequencing (RRBS) profiling of 843 cell lines, which measures methylation levels at CpG sites across the genome. We aggregated these CpG-level estimates to look at methylation of gene promoter regions, promoter-proximal CpG clusters, and enhancer segments. 

## Promoter regions (1kb ahead of the TSS)

In [ ]:
tss1kb_meth = pd.read_csv("../data/CCLE_RRBS_TSS1kb_20181022.txt",sep="\t",index_col=0)

# Drop info columns
tss1kb_meth = tss1kb_meth.iloc[:-1,2:]
tss1kb_meth = tss1kb_meth.T

# Achilles name map
tss1kb_meth.index = tss1kb_meth.index.map(lambda x: ccle_achilles_map[x])

# Fix inconsistant NaN values
tss1kb_meth[tss1kb_meth=="\tNA"] = np.nan
tss1kb_meth[tss1kb_meth=="    NA"] = np.nan
tss1kb_meth[tss1kb_meth=="     NA"] = np.nan
tss1kb_meth = tss1kb_meth.astype(float)

# Standard deviation filtering
tss1kb_meth_stds = tss1kb_meth.std(axis=0)
sns.distplot(tss1kb_meth_stds)
tss1kb_meth = tss1kb_meth[tss1kb_meth.columns[tss1kb_meth_stds>0.05]]

# Export to HDF5
tss1kb_meth.to_hdf("../data/CCLE_RRBS_TSS1kb_20181022.hdf",key="tss1kb_meth")

## Promoter region clusters

In [ ]:
tssclust_meth = pd.read_csv("../data/CCLE_RRBS_tss_CpG_clusters_20181022.txt",sep="\t",index_col=0)

# Drop info columns
tssclust_meth = tssclust_meth.iloc[:-1,2:]
tssclust_meth = tssclust_meth.T

# Achilles name map
tssclust_meth.index = tssclust_meth.index.map(lambda x: ccle_achilles_map[x])

# Fix inconsistant NaN values
tssclust_meth[tssclust_meth=="\tNA"] = np.nan
tssclust_meth[tssclust_meth=="    NA"] = np.nan
tssclust_meth[tssclust_meth=="     NA"] = np.nan
tssclust_meth = tssclust_meth.astype(float)

# Standard deviation filtering
tssclust_meth_stds = tssclust_meth.std(axis=0)
sns.distplot(tssclust_meth_stds)
tssclust_meth = tssclust_meth[tssclust_meth.columns[tssclust_meth_stds>0.05]]

# Export to HDF5
tssclust_meth.to_hdf("../data/CCLE_RRBS_tss_CpG_clusters_20181022.hdf",key="tssclust_meth")

# miRNA profiling

In [ ]:
mirna = pd.read_csv("../data/CCLE_miRNA_20181103.gct.txt",sep="\t",skiprows=2)
mirna.index = mirna["Description"] + "_" + mirna["Name"].apply(lambda x: x[1:])

# Drop info columns and log2-transform
mirna = mirna.iloc[:,2:]
mirna = np.log2(mirna.T)

# Achilles name map
mirna.index = mirna.index.map(lambda x: ccle_achilles_map[x])

# Export to HDF5
mirna.to_hdf("../data/CCLE_miRNA_20181103.hdf",key="mirna")

# Mutation profiling

In [20]:
mutation_calls = pd.read_csv("../data/depmap_19Q1_mutation_calls.csv",index_col=0)

damaging_muts = mutation_calls[mutation_calls["Variant_annotation"]=="damaging"]
hs_muts = mutation_calls[(mutation_calls["isCOSMIChotspot"]==True)|(mutation_calls["isTCGAhotspot"]==True)]

# Counts of each mutation
damaging_counts = Counter(damaging_muts["Hugo_Symbol"])
hs_counts = Counter(hs_muts["Hugo_Symbol"])

damaging_muts["count"] = damaging_muts["Hugo_Symbol"].apply(lambda x: damaging_counts[x])
hs_muts["count"] = hs_muts["Hugo_Symbol"].apply(lambda x: hs_counts[x])

# Keep recurrently mutated genes
damaging_muts = damaging_muts[damaging_muts["count"]>=8]
hs_muts = hs_muts[hs_muts["count"]>=8]

damaging_muts["id"] = damaging_muts["Hugo_Symbol"] + "_" + damaging_muts["DepMap_ID"]
hs_muts["id"] = hs_muts["Hugo_Symbol"] + "_" + hs_muts["DepMap_ID"]

# Drop double-mutated instances
damaging_muts = damaging_muts.drop_duplicates(subset=["id"],keep="first")
hs_muts = hs_muts.drop_duplicates(subset=["id"],keep="first")

# Dummy value for pivoting
hs_muts["value"] = 1
damaging_muts["value"] = 1

# Pivot from list of cell lines + mutations to cell lines vs. mutations
hs_mut_mat = pd.pivot_table(hs_muts, values="value", index=[
                            "DepMap_ID"], columns="Hugo_Symbol", fill_value=0)
damaging_mut_mat = pd.pivot_table(damaging_muts, values="value", index=[
                                  "DepMap_ID"], columns="Hugo_Symbol", fill_value=0)

# Export to HDF5
hs_mut_mat.to_hdf("../data/hs_muts.h5",key="hs_muts")
damaging_mut_mat.to_hdf("../data/damaging_muts.h5",key="damaging_muts")

/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/khu/anaconda3/envs/cenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
